## 11: Monkey in the Middle

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day11/puzzle.ipynb)

### Parsing

In [118]:
#!value --name sampleRaw
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1

In [119]:
type Monkey = { 
    Items: int64 list
    Operation: int64 -> int64
    Activity: int64
    TestDivisible: int64
    TestTrueToMonkey: int
    TestFalseToMonkey: int
}
type State = { Monkeys: Map<int,Monkey> }


In [120]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder

#load "../common/common.fsx"

Installed Packages Farkle, 6.3.2

In [121]:
let private number = Terminals.int64 "Number"
let private monkeyId = "Id" ||= [
    !& "Monkey" .>>. number .>> ":" |> asIs
]
let private items = "Items" ||= [
    !& "Starting items:" .>>. (sepBy1 (literal ",") number) |> asIs
]
let private oldRef = literal "old"
let private operator = "Operator" ||= [
    !& "+" =% (+)
    !& "*" =% (*)
]
let private operand = "Operand" ||= [
    !% oldRef =% id
    !@ number => (fun num -> fun  (old:int64) -> num)
]
let private operation = "Operation" ||= [
    !& "Operation: new =" .>>. operand .>>. operator .>>. operand 
        => (fun x f y -> fun (old:int64) -> f (x old) (y old))
]
let private divisible = "TestDivisible" ||= [
    !& "Test: divisible by" .>>. number |> asIs
]
let private trueMonkey = "TestTrueToMonkey" ||= [
    !& "If true: throw to monkey" .>>. number |> asIs
]
let private falseMonkey = "TestFalseToMonkey" ||= [
    !& "If false: throw to monkey" .>>. number |> asIs
]
let private monkey = "Monkey" ||= [
    !@ monkeyId .>>. items .>>. operation .>>. divisible .>>. trueMonkey .>>. falseMonkey
        => fun id items operation divisible trueMonkey falseMonkey -> 
            int id, {
                Items = items
                Activity = 0L
                Operation = operation
                TestDivisible = divisible
                TestTrueToMonkey = int trueMonkey
                TestFalseToMonkey = int falseMonkey
            }
]

let private parser = RuntimeFarkle.build monkey

let parseMonkey s = 
    s |> RuntimeFarkle.parseString parser |> Result.get

In [122]:
#!share sampleRaw --from value

let sampleMonkeys = 
    sampleRaw |> Pattern2.read parseMonkey
    |> Map.ofSeq

sampleMonkeys

key Items Operation Activity TestDivisible TestTrueToMonkey TestFalseToMonkey 0 [ 79, 98 ] f t Invoke@3584-1
 f: Invoke@3623-2
 f: FSI_0244+operation@19-23
 t: FSI_0244+operand@14-46
 t: FSI_0244+operator@11-47 Invoke@3570
 f: FSI_0244+operand@15-47
 t: 19 0 23 2 3 1 [ 54, 65, 75, 74 ] f t Invoke@3584-1
 f: Invoke@3623-2
 f: FSI_0244+operation@19-23
 t: FSI_0244+operand@14-46
 t: FSI_0244+operator@10-46 Invoke@3570
 f: FSI_0244+operand@15-47
 t: 6 0 19 2 0 2 [ 79, 60, 97 ] f t Invoke@3584-1
 f: Invoke@3623-2
 f: FSI_0244+operation@19-23
 t: FSI_0244+operand@14-46
 t: FSI_0244+operator@11-47 FSI_0244+operand@14-46 0 13 1 3 3 [ 74 ] f t Invoke@3584-1
 f: Invoke@3623-2
 f: FSI_0244+operation@19-23
 t: FSI_0244+operand@14-46
 t: FSI_0244+operator@10-46 Invoke@3570
 f: FSI_0244+operand@15-47
 t: 3 0 17 0 1

### Part 1

For part 1 we are going to skip the condition of devision by three in common functions and tweak the monkeys' operations instead.

In [123]:

let roundMonkey i (s : State) =
    let m = s.Monkeys[i]
    let newItems = 
        m.Items 
        |> List.map m.Operation

    let (newItemsT, newItemsF) = 
        newItems
        |> List.rev
        |> List.partition (fun item -> item % m.TestDivisible = 0)

    let newM = { m with Items = []; Activity = m.Activity + int64 (List.length newItems) }

    {
        Monkeys = 
            s.Monkeys
            |> Map.change i (fun x -> Some newM)
            |> Map.change m.TestTrueToMonkey (fun x -> Some { x.Value with Items = x.Value.Items @ newItemsT })
            |> Map.change m.TestFalseToMonkey (fun x -> Some { x.Value with Items = x.Value.Items @ newItemsF })
    }
(*
    match m.Items with 
    | [] -> s 
    | item::rest -> 
        
        let newItem = (m.Operation (item))
        let newMonkeyIndex = 
            if (newItem % m.TestDivisible = 0)
            then m.TestTrueToMonkey
            else m.TestFalseToMonkey
        let newM = { m with Items = rest; Activity = m.Activity + 1L }
        let newState = 
            {
                Monkeys = 
                    s.Monkeys
                    |> Map.change newMonkeyIndex (fun x -> Some { x.Value with Items = x.Value.Items @ [newItem]})
                    |> Map.change i (fun x -> Some newM)
            }
        newState :: (roundMonkey i newState)*)
let round (s:State) =
    s.Monkeys
    |> Map.toSeq
    |> Seq.sortBy fst
    |> Seq.fold (fun previous (i, m) -> roundMonkey i previous) s

In [124]:
module Part1 = 
    let tweakOperations (state : State) = 
        {state with 
            Monkeys = 
                state.Monkeys
                |> Map.map (fun key m ->
                    { m with Operation = fun old -> m.Operation old / 3L }
                ) 
        }
    let solveAndPrint (state : State) = 
        let rounds = [0..19]
        (rounds
            |> List.fold (fun previous _ -> round previous) (tweakOperations state)
        ).Monkeys
        |> Map.map (fun key m -> m.Activity)
        |> displayPipe
        |> Map.values
        |> Seq.sortDescending |> Seq.take 2 |> Seq.reduce (*)

{ Monkeys = sampleMonkeys } |> Part1.solveAndPrint

key,value
0,101
1,95
2,7
3,105


10605

In [125]:
#!value --name actualRaw --from-file ./data_actual.txt

In [126]:
#!share actualRaw --from value
let actualMonkeys =
    actualRaw |> Pattern2.read parseMonkey
    |> Map.ofSeq
{ Monkeys = actualMonkeys } |> Part1.solveAndPrint

key,value
0,225
1,16
2,236
3,158
4,69
5,12
6,232
7,246


58056

### Part 2

Apparentely, 64 bits won't be enough to store the worry levels. We can notice that we do not need the actual values though, but the moduli. And luckily, all the divisors are different prime numbers. So, given `commonDivisible` is as a production of them, whatever set of operations a certain item went through, the result $X$ of these operations will satisfy the following equation for any monkey:
$$X\ \mathrm{mod}\ {TestDivisible} = \left(X\ \mathrm{mod}\ {commonDivisible}\right)\ \mathrm{mod}\ {TestDivisible} \qquad {\Large \forall}\;  TestDivisible$$ 

, while only additions and productions are possible.

Like in Part 1, modulo operation can be a tweak of each operation from the input.

In [129]:
module Part2 = 
    let tweakOperations (state : State) = 
        let commonDivisible =
            state.Monkeys.Values
            |> Seq.map (fun f -> f.TestDivisible)
            |> Seq.reduce (*)
        
        {state with 
            Monkeys = 
                state.Monkeys
                |> Map.map (fun key m ->
                    { m with Operation = fun old -> m.Operation old % commonDivisible }
                ) 
        }
    let solveAndPrint (state : State) = 
        let rounds = [0..9999]
        (rounds
            |> List.fold (fun previous _ -> round previous) (tweakOperations state)
        ).Monkeys
        |> Map.map (fun key m -> m.Activity)
        |> displayPipe
        |> Map.values
        |> Seq.sortDescending |> Seq.take 2 |> Seq.reduce (*)

{ Monkeys = sampleMonkeys } |> Part2.solveAndPrint

key,value
0,52166
1,47830
2,1938
3,52013


2713310158

In [130]:
{ Monkeys = actualMonkeys } |> Part2.solveAndPrint

key,value
0,122937
1,20445
2,122291
3,62033
4,32278
5,1538
6,122410
7,122181


15048718170